In [ ]:
!pip install -q -U transformers einops accelerate peft xformers langchain safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Due to long inputs, we will not use the model here. We will create the input and the example(s) selector and find the selected examples. Then, we will use these results in ChainForge to generate the final description.

In [ ]:
from langchain import PromptTemplate

multi_input_template = """
You are an expert in fashion design. Write a description for a fashion outfit in {style} style appropriate for a {role} at a {occasion}. Be sure to address the colors and the textures.
"""

multi_input_prompt = PromptTemplate(input_variables=["style", "role", "occasion"], template=multi_input_template)

In [ ]:
multi_input_prompt.format(style="bohemian", role="a petite apple shaped woman", occasion="wedding")

'\nYou are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a a petite apple shaped woman at a wedding. Be sure to address the colors and the textures.\n'

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Read CSV file
simple_types = "simple_types.csv"  # Provide the path to your CSV file
simple_types_df = pd.read_csv(simple_types)
simple_types_df

,Style,Type,Event
0,bohemian,man,a casual date
1,bohemian,woman,a halloween party
2,classic,man,a bar
3,classic,woman,a work / office event
4,gothic,man,a work / office event
...,...,...,...
95,gothic,woman,a business lunch
96,sporty,man,a conference
97,sporty,woman,a beach
98,casual,man,a halloween party


In [ ]:
# Initialize a list to store results
inputs = []

for index, row in simple_types_df.iterrows():
    style = row['Style']
    role = row['Type']
    occasion = row['Event']
    # Generate outfit description
    input = multi_input_prompt.format(style=style, role=role, occasion=occasion)
    inputs.append({'style': style, 'role': role, 'occasion': occasion, 'input': input})

In [ ]:
# Convert list of results to DataFrame
inputs_df = pd.DataFrame(inputs)
inputs_df

,style,role,occasion,input
0,bohemian,man,a casual date,\nYou are an expert in fashion design. Write a...
1,bohemian,woman,a halloween party,\nYou are an expert in fashion design. Write a...
2,classic,man,a bar,\nYou are an expert in fashion design. Write a...
3,classic,woman,a work / office event,\nYou are an expert in fashion design. Write a...
4,gothic,man,a work / office event,\nYou are an expert in fashion design. Write a...
...,...,...,...,...
95,gothic,woman,a business lunch,\nYou are an expert in fashion design. Write a...
96,sporty,man,a conference,\nYou are an expert in fashion design. Write a...
97,sporty,woman,a beach,\nYou are an expert in fashion design. Write a...
98,casual,man,a halloween party,\nYou are an expert in fashion design. Write a...


Here, we have all our inputs in column "input". Let's create an examples database

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
examples = [
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual style appropriate for a man at a concert. Be sure to address the colors and the textures."
    ,
    "answer": "For a man at a concert, envision a sleek black leather jacket over distressed skinny jeans. Combine it with an oversized charcoal graphic tee for an edgy but classic touch. Complete the look with black ankle boots, silver accessories, and tousled waves for a cool and confident stage presence."
  },
  {"question": " You are an expert in fashion design. Write a description for a fashion outfit in preppy style appropriate for a man at a summer concert. Be sure to address the colors and the textures.",
  "answer": "Imagine a crisp white Oxford shirt paired with light khaki chino shorts for a classic preppy look. Layer it with a navy blue V-neck sweater draped over the shoulders for a touch of sophistication. Complete the ensemble with brown leather boat shoes and accessorize with a woven brown belt and a pair of tortoiseshell sunglasses for a polished summer vibe."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in minimalist style appropriate for a woman at an indie concert. Be sure to address the colors and the textures.",
  "answer": "For a minimalist look, opt for a structured black sleeveless midi dress crafted from smooth crepe fabric. Pair it with sleek black ankle boots with a chunky heel for added edge. Keep accessories simple with silver hoop earrings and a black leather crossbody bag, allowing the clean lines and monochromatic palette to make a statement."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in athleisure style appropriate for a man at a music festival. Be sure to address the colors and the textures.",
  "answer": "Envision a cozy oversized sweatshirt in heather gray, layered over black performance joggers with contrasting side stripes for a sporty vibe. Pair it with white chunky sneakers for comfort and style. Accessorize with a black baseball cap and reflective sunglasses for an athletic yet trendy look perfect for a music festival."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in vintage style appropriate for a woman at a retro concert. Be sure to address the colors and the textures.",
  "answer": "Picture a retro-inspired floral tea dress in a vibrant print of bold colors and swirling patterns. Pair it with chunky platform sandals in a complementary color for a touch of retro flair. Layer with a cropped denim jacket for a casual, vintage vibe. Accessorize with cat-eye sunglasses and a woven straw handbag to complete the look."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in eclectic style appropriate for a man at a music festival. Be sure to address the colors and the textures.",
  "answer": "For an eclectic look, combine a statement graphic T-shirt featuring bold colors and abstract designs with distressed denim jeans for a casual yet eye-catching ensemble. Layer with a lightweight kimono-style jacket in a vibrant pattern for added flair. Complete the look with suede Chelsea boots and accessorize with beaded bracelets and a fedora hat for a bohemian-inspired vibe."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in edgy style appropriate for a woman at a rock concert. Be sure to address the colors and the textures.",
  "answer": "Envision a faux leather moto jacket in sleek black, layered over a graphic band tee for a rebellious edge. Pair it with distressed black skinny jeans for a rock 'n' roll vibe. Complete the look with studded ankle boots and accessorize with silver chain necklaces and hoop earrings for an edgy yet feminine ensemble."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in retro style appropriate for a man at a vintage concert. Be sure to address the colors and the textures.",
  "answer": "Imagine a retro-inspired Hawaiian shirt in a vibrant print of tropical flowers and palm leaves, paired with high-waisted beige chino shorts for a classic vintage look. Layer it with a lightweight beige linen blazer for added sophistication. Complete the ensemble with woven espadrilles and accessorize with retro sunglasses and a straw fedora hat for a nod to nostalgia."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual chic style appropriate for a woman at an outdoor concert. Be sure to address the colors and the textures.",
  "answer": "Picture a breezy off-the-shoulder blouse in crisp white cotton, paired with high-waisted denim shorts for a relaxed yet chic look. Layer with a lightweight beige linen cardigan for cooler evenings. Complete the ensemble with tan leather sandals and accessorize with a woven straw tote bag and oversized sunglasses for effortless summer style."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in boho style appropriate for a woman at a folk concert. Be sure to address the colors and the textures.",
  "answer": "Envision a flowy maxi skirt in a colorful bohemian print, paired with a crochet crop top for a free-spirited vibe. Layer with a distressed denim jacket for a casual touch. Complete the look with suede ankle boots and accessorize with layered beaded necklaces and a fringe crossbody bag for a boho-chic ensemble."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual elegance style appropriate for a man at an acoustic concert. Be sure to address the colors and the textures.",
  "answer": "For a casual yet elegant look, opt for a tailored navy blazer paired with a crisp white button-down shirt and beige chinos for a timeless ensemble. Layer with a lightweight knit sweater in a complementary shade for added warmth. Complete the look with brown leather loafers and accessorize with a leather belt and classic wristwatch for understated sophistication."
  }
  ,
   {"question": "You are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a woman at an outdoor concert. Be sure to address the colors and the textures.",
  "answer": "Picture a flowy floral maxi dress in earthy tones of mustard yellow and olive green, with delicate ruffle detailing and a tie-waist accentuating the figure. Layer it with a crochet-knit cardigan in a soft ivory hue for a bohemian touch. Complete the look with suede ankle boots in a rich cognac shade and accessorize with layered beaded bracelets and a wide-brimmed floppy hat for a free-spirited vibe."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual streetwear style appropriate for a teenager at a hip-hop concert. Be sure to address the colors and the textures.",
  "answer": "Imagine an oversized graphic hoodie in a bold neon color, layered over distressed black skinny jeans for a streetwear edge. Pair it with chunky white sneakers featuring contrasting black accents for a modern twist. Add a black snapback cap with embroidered logo detailing and accessorize with a silver chain necklace and sporty wristwatch for an urban-cool look."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in urban chic style appropriate for a tall man with a slim build at a music festival. Be sure to address the colors and the textures.",
  "answer": "For the tall, slim man at a music festival, envision a fitted black leather biker jacket over a vintage-inspired band tee featuring bold graphics and distressed detailing. Pair it with slim-cut charcoal gray jeans for a modern, monochromatic look. Add a pop of color with vibrant red high-top sneakers, and accessorize with a chunky silver chain necklace and a sleek leather wristwatch for urban flair."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a petite woman with a curvy figure at an outdoor concert. Be sure to address the colors and the textures.",
  "answer": "For the petite woman with a curvy figure at an outdoor concert, picture a flowy floral maxi dress in earthy tones of mustard yellow and olive green, with delicate ruffle detailing and a tie-waist accentuating the figure. Layer it with a crochet-knit cardigan in a soft ivory hue for a bohemian touch. Complete the look with suede ankle boots in a rich cognac shade and accessorize with layered beaded bracelets and a wide-brimmed floppy hat for a free-spirited vibe."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual streetwear style appropriate for a tall man with a slender build at a hip-hop concert. Be sure to address the colors and the textures.",
  "answer": "Imagine an oversized graphic hoodie in a bold neon color, layered over distressed black skinny jeans for a streetwear edge, perfect for the tall man with a slender build. Pair it with chunky white sneakers featuring contrasting black accents for a modern twist. Add a black snapback cap with embroidered logo detailing and accessorize with a silver chain necklace and sporty wristwatch for an urban-cool look."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in vintage style appropriate for a petite woman with an hourglass figure at a retro concert. Be sure to address the colors and the textures.",
  "answer": "Imagine a retro-inspired floral tea dress in a vibrant print of bold colors and swirling patterns, perfect for the petite woman with an hourglass figure. Pair it with chunky platform sandals in a complementary color for a touch of retro flair. Layer with a cropped denim jacket for a casual, vintage vibe. Accessorize with cat-eye sunglasses and a woven straw handbag to complete the look."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in eclectic style appropriate for a tall man with a lean build at a music festival. Be sure to address the colors and the textures.",
  "answer": "For the tall man with a lean build at a music festival, combine a statement graphic T-shirt featuring bold colors and abstract designs with distressed denim jeans for a casual yet eye-catching ensemble. Layer with a lightweight kimono-style jacket in a vibrant pattern for added flair. Complete the look with suede Chelsea boots and accessorize with beaded bracelets and a fedora hat for a bohemian-inspired vibe."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in edgy style appropriate for a curvy woman with a petite frame at a rock concert. Be sure to address the colors and the textures.",
  "answer": "Envision a faux leather moto jacket in sleek black, layered over a graphic band tee for a rebellious edge, perfect for the curvy woman with a petite frame. Pair it with distressed black skinny jeans for a rock 'n' roll vibe. Complete the look with studded ankle boots and accessorize with silver chain necklaces and hoop earrings for an edgy yet feminine ensemble."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in retro style appropriate for a tall man with a slender build at a vintage concert. Be sure to address the colors and the textures.",
  "answer": "Imagine a retro-inspired Hawaiian shirt in a vibrant print of tropical flowers and palm leaves, paired with high-waisted beige chino shorts for a classic vintage look, tailored for the tall man with a slender build. Layer it with a lightweight beige linen blazer for added sophistication. Complete the ensemble with woven espadrilles and accessorize with retro sunglasses and a straw fedora hat for a nod to nostalgia."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual chic style appropriate for a petite woman with a curvy figure at an outdoor concert. Be sure to address the colors and the textures.",
  "answer": "Picture a breezy off-the-shoulder blouse in crisp white cotton, paired with high-waisted denim shorts for a relaxed yet chic look, tailored for the petite woman with a curvy figure. Layer with a lightweight beige linen cardigan for cooler evenings. Complete the ensemble with tan leather sandals and accessorize with a woven straw tote bag and oversized sunglasses for effortless summer style."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in boho style appropriate for a tall woman with a slender build at a folk concert. Be sure to address the colors and the textures.",
  "answer": "Envision a flowy maxi skirt in a colorful bohemian print, paired with a crochet crop top for a free-spirited vibe, perfect for the tall woman with a slender build. Layer with a distressed denim jacket for a casual touch. Complete the look with suede ankle boots and accessorize with layered beaded necklaces and a fringe crossbody bag for a boho-chic ensemble."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual elegance style appropriate for a tall man with a lean build at an acoustic concert. Be sure to address the colors and the textures.",
  "answer": "For the tall man with a lean build at an acoustic concert, opt for a tailored navy blazer paired with a crisp white button-down shirt and beige chinos for a timeless ensemble. Layer with a lightweight knit sweater in a complementary shade for added warmth. Complete the look with brown leather loafers and accessorize with a leather belt and classic wristwatch for understated sophistication."
  },
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual bohemian style appropriate for a tall woman with a slender build at a music festival. Be sure to address the colors and the textures.",
  "answer": "Envision a flowy peasant blouse in a soft pastel shade, paired with high-waisted denim shorts adorned with intricate embroidery for a bohemian touch, perfect for the tall woman with a slender build. Layer with a crochet vest in a complementary hue for added texture. Complete the look with brown leather sandals and accessorize with layered pendant necklaces and a fringed crossbody bag for a carefree festival vibe."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual streetwear style appropriate for a petite man with a muscular build at a hip-hop concert. Be sure to address the colors and the textures.",
  "answer": "Imagine a fitted graphic T-shirt featuring bold colors and urban-inspired designs, paired with distressed black jogger pants for a streetwear edge, tailored for the petite man with a muscular build. Layer with a zip-up hoodie in a contrasting color for added depth. Complete the look with high-top sneakers in a vibrant hue and accessorize with a baseball cap worn backwards and chunky chain necklace for an urban-cool ensemble."
  }
  ,
  {"question": "You are an expert in fashion design. Write a description for a fashion outfit in casual elegance style appropriate for a curvy woman with a petite frame at an outdoor concert. Be sure to address the colors and the textures.",
  "answer": "Picture a draped wrap dress in a rich jewel tone, cinched at the waist to accentuate curves and tailored for the curvy woman with a petite frame. Layer with a lightweight cashmere cardigan in a complementary shade for added warmth and sophistication. Complete the ensemble with nude patent leather pumps and accessorize with delicate gold jewelry and a clutch purse for understated elegance."
  }
]

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\nAnswer: {answer}"
)

print(example_prompt.format(**examples[0]))

Question: You are an expert in fashion design. Write a description for a fashion outfit in casual style appropriate for a man at a concert. Be sure to address the colors and the textures.
Answer: For a man at a concert, envision a sleek black leather jacket over distressed skinny jeans. Combine it with an oversized charcoal graphic tee for an edgy but classic touch. Complete the look with black ankle boots, silver accessories, and tousled waves for a cool and confident stage presence.


In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
from langchain_core.example_selectors.base import BaseExampleSelector
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        # This assumes knowledge that part of the input will be a 'text' key
        new_word = input_variables["input"]
        # new_word_length = len(new_word)
        new_word_emb = model.encode(new_word, convert_to_tensor=True)

        # Initialize variables to store the best match and its length difference
        best_match = None
        minimum_cos = -1

        # Iterate through each example
        for example in self.examples:
            example_emb = model.encode(example["question"], convert_to_tensor=True)
            # Calculate the length difference with the first word of the example
            cosine_similarity = util.pytorch_cos_sim(new_word_emb, example_emb)


            # Update the best match if the current one is closer in length
            if cosine_similarity > minimum_cos:
                minimum_cos = cosine_similarity
                best_match = example

        return [best_match]

In [ ]:
example_selector = CustomExampleSelector(examples)

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="\nQuestion: {input}\nAnswer:",
    input_variables=["input"],
)

In [ ]:
# Assuming 'inputs_df' is your DataFrame and 'prompt' is your format string
new_inputs_list = []

for input_value in inputs_df['input']:
    new_input = prompt.format(input= input_value)
    new_inputs_list.append(new_input)

print(new_inputs_list)


In [ ]:
# Convert list of results to DataFrame
new_inputs_df = pd.DataFrame(new_inputs_list)
new_inputs_df

In [ ]:
# Specify the output CSV file name
output_csv_file = "few-shot_simple-types_inputs-with-examples.csv"

# Save DataFrame to CSV file
new_inputs_df.to_csv(output_csv_file, index=False)

Now, let's repeat for the complex types


In [ ]:
# Read CSV file
complex_types = "complex_types.csv"  # Provide the path to your CSV file
complex_types_df = pd.read_csv(complex_types)
complex_types_df

,Style,Type,Event
0,bohemian,"a small-framed, delicate woman",a work / office event
1,bohemian,"a short, curvy man with a muscular build",a casual date
2,classic,"a small-framed, delicate woman",a play / concert
3,classic,"a short, curvy man with a muscular build",a work / office event
4,gothic,"a small-framed, delicate woman",a beach
...,...,...,...
95,gothic,"a short, curvy man with a muscular build",a birthday party
96,sporty,"a small-framed, delicate woman",a wedding
97,sporty,"a short, curvy man with a muscular build",a night club
98,casual,"a small-framed, delicate woman",a business meeting


In [ ]:
# Initialize a list to store results
complex_inputs = []

for index, row in complex_types_df.iterrows():
    style = row['Style']
    role = row['Type']
    occasion = row['Event']
    # Generate outfit description
    input = multi_input_prompt.format(style=style, role=role, occasion=occasion)
    complex_inputs.append({'style': style, 'role': role, 'occasion': occasion, 'input': input})

In [ ]:
# Convert list of results to DataFrame
complex_inputs_df = pd.DataFrame(complex_inputs)
complex_inputs_df

,style,role,occasion,input
0,bohemian,"a small-framed, delicate woman",a work / office event,\nYou are an expert in fashion design. Write a...
1,bohemian,"a short, curvy man with a muscular build",a casual date,\nYou are an expert in fashion design. Write a...
2,classic,"a small-framed, delicate woman",a play / concert,\nYou are an expert in fashion design. Write a...
3,classic,"a short, curvy man with a muscular build",a work / office event,\nYou are an expert in fashion design. Write a...
4,gothic,"a small-framed, delicate woman",a beach,\nYou are an expert in fashion design. Write a...
...,...,...,...,...
95,gothic,"a short, curvy man with a muscular build",a birthday party,\nYou are an expert in fashion design. Write a...
96,sporty,"a small-framed, delicate woman",a wedding,\nYou are an expert in fashion design. Write a...
97,sporty,"a short, curvy man with a muscular build",a night club,\nYou are an expert in fashion design. Write a...
98,casual,"a small-framed, delicate woman",a business meeting,\nYou are an expert in fashion design. Write a...


In [ ]:
# Assuming 'inputs_df' is your DataFrame and 'prompt' is your format string
new_complex_inputs_list = []

for input_value in complex_inputs_df['input']:
    new_input = prompt.format(input= input_value)
    new_complex_inputs_list.append(new_input)

print(new_complex_inputs_list)


['Question: You are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a woman at an outdoor concert. Be sure to address the colors and the textures.\nAnswer: Picture a flowy floral maxi dress in earthy tones of mustard yellow and olive green, with delicate ruffle detailing and a tie-waist accentuating the figure. Layer it with a crochet-knit cardigan in a soft ivory hue for a bohemian touch. Complete the look with suede ankle boots in a rich cognac shade and accessorize with layered beaded bracelets and a wide-brimmed floppy hat for a free-spirited vibe.\n\n\nQuestion: \nYou are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a a small-framed, delicate woman at a a work / office event. Be sure to address the colors and the textures.\n\nAnswer:', 'Question: You are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a petite wo

In [ ]:
# Convert list of results to DataFrame
new_complex_inputs_list_df = pd.DataFrame(new_complex_inputs_list)
new_complex_inputs_list_df

,0
0,Question: You are an expert in fashion design....
1,Question: You are an expert in fashion design....
2,Question: You are an expert in fashion design....
3,Question: You are an expert in fashion design....
4,Question: You are an expert in fashion design....
...,...
95,Question: You are an expert in fashion design....
96,Question: You are an expert in fashion design....
97,Question: You are an expert in fashion design....
98,Question: You are an expert in fashion design....


In [ ]:
# Specify the output CSV file name
output_csv_file = "few-shot_complex-types_inputs-with-examples.csv"

# Save DataFrame to CSV file
new_complex_inputs_list_df.to_csv(output_csv_file, index=False)

Now, let's create a df that includes the variables and the new descriptions.


In [ ]:
simple_types_final_concatenated_df = pd.concat([inputs_df, new_inputs_df], axis=1)
complex_types_final_concatenated_df = pd.concat([complex_inputs_df, new_complex_inputs_list_df], axis=1)

In [ ]:
simple_types_final_concatenated_df

,style,role,occasion,input,0
0,bohemian,man,a casual date,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
1,bohemian,woman,a halloween party,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
2,classic,man,a bar,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
3,classic,woman,a work / office event,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
4,gothic,man,a work / office event,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
...,...,...,...,...,...
95,gothic,woman,a business lunch,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
96,sporty,man,a conference,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
97,sporty,woman,a beach,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
98,casual,man,a halloween party,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....


In [ ]:
complex_types_final_concatenated_df

,style,role,occasion,input,0
0,bohemian,"a small-framed, delicate woman",a work / office event,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
1,bohemian,"a short, curvy man with a muscular build",a casual date,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
2,classic,"a small-framed, delicate woman",a play / concert,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
3,classic,"a short, curvy man with a muscular build",a work / office event,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
4,gothic,"a small-framed, delicate woman",a beach,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
...,...,...,...,...,...
95,gothic,"a short, curvy man with a muscular build",a birthday party,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
96,sporty,"a small-framed, delicate woman",a wedding,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
97,sporty,"a short, curvy man with a muscular build",a night club,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....
98,casual,"a small-framed, delicate woman",a business meeting,\nYou are an expert in fashion design. Write a...,Question: You are an expert in fashion design....


In [ ]:
# Specify the output CSV file name
output_csv_file = "few-shot_simple-types_inputs-with-examples-and-variables.csv"

# Save DataFrame to CSV file
simple_types_final_concatenated_df.to_csv(output_csv_file, index=False)

In [ ]:
# Specify the output CSV file name
output_csv_file = "few-shot_complex-types_inputs-with-examples-and-variables.csv"

# Save DataFrame to CSV file
complex_types_final_concatenated_df.to_csv(output_csv_file, index=False)